In [1]:
import json
import pandas as pd
import spacy
import html.parser
import re
from flashgeotext.geotext import GeoText
geotext = GeoText(use_demo_data=True)

nlpEn = spacy.load("en_core_web_lg") 
nlpFR = spacy.load('fr_core_news_md')
nlpsm = spacy.load("en_core_web_sm")
nlpMulti = spacy.load("xx_ent_wiki_sm") 

2020-09-11 00:01:52.213 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-09-11 00:01:52.217 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-09-11 00:01:52.218 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


In [2]:
from spacy import displacy

In [3]:
df = pd.read_json('data/dgOutput1000.json')
df.columns


Index(['bidDeadlineDate', 'buyerCountry', 'buyerName', 'contactAddress',
       'contactCity', 'contactCountry', 'contactEmail', 'contactFirstName',
       'contactLastName', 'contactPhone', 'contactState', 'country', 'cpvs',
       'creationDate', 'currency', 'description', 'dgmarketId',
       'estimatedCost', 'exclusion', 'exclusionWord', 'fileSource', 'lang',
       'noticeType', 'procurementId', 'procurementMethod', 'publicationDate',
       'sourceUrl', 'status', 'title', 'updateDate'],
      dtype='object')

In [4]:
def clean(Texte_Concat, accents = True):
    junk_space = [\
                    '<[^<]+?>', '\r\n', '!', ',', ';', '  ', '\t'\
                    '.', '?', '(', ')', '[', ']', '{', '}', '%', '*', '-', '_', '•', \
                    ':', '#', '€','–','>','<','«','»','+', '"', 'n°','N°', '●', '/', \
                    '►', '\n', '=', '“', '”', '…', '→', '‹', '≥', '⋅', '○',  \
                     '’', '‘', '—', '‑', 'Ø', 'Ä' \
                    ]
    Texte_without_junk = html.parser.unescape(Texte_Concat)
    Texte_without_junk = re.sub('<[^<]+?>', '', Texte_without_junk)

    for elmt in junk_space:
        Texte_without_junk = Texte_without_junk.replace(elmt,  ' ')
    return Texte_without_junk

In [5]:
df["text_clean"] = df["description"].apply(clean)

In [6]:
test = df["text_clean"][3]
test

"Contract notice  Hosting of the is of the anact aract network.  The service includes  hosting and monitoring of physical environments and production and pre production software operations management  supervision backups restorations archiving data security incident resolution co  administration of certain elements of the technical infrastructure supply of materials necessary for the performance of the contract associated services  advice technical support and engineering.  This contract is divided into lots  no  Deadline for receipt of tenders or requests to participate  September 18 2020   5 00 pm  Language  s  official  s  of the EU  French  Avis No 20 100581   publié le 09 08 2020   Dépt.   69 Anact departement finance gestion   Hébergement du SI du réseau Anact Aract.   Avis de marché Procédure Ouverte        Références complètes&nbsp     Avis  20 100581        Référence de TED     non disponible         Annonce            Département s  de publication   69Annonce No 20 100581I.II

In [7]:
#geoRes = geotext.extract(input_text= test, span_info=True)["cities"]
#geoRes
def traitement(x):
    cities = geotext.extract(input_text= x, span_info=True)["cities"]
    return(cities)
df['cities'] = df['description'].apply(traitement)

In [8]:
df['cities'] 

0                                                     {}
1      {'Saint-Denis': {'count': 1, 'span_info': [(33...
2      {'Nanterre': {'count': 2, 'span_info': [(1831,...
3      {'Lyon': {'count': 1, 'span_info': [(2301, 230...
4      {'Nanterre': {'count': 2, 'span_info': [(2464,...
5      {'Valenciennes': {'count': 3, 'span_info': [(2...
6      {'Mission': {'count': 1, 'span_info': [(1297, ...
7        {'Of': {'count': 1, 'span_info': [(112, 114)]}}
8      {'David': {'count': 1, 'span_info': [(288, 293...
9      {'Date': {'count': 1, 'span_info': [(880, 884)...
10     {'Date': {'count': 1, 'span_info': [(880, 884)...
11     {'Date': {'count': 1, 'span_info': [(880, 884)...
12     {'Date': {'count': 1, 'span_info': [(880, 884)...
13     {'Date': {'count': 1, 'span_info': [(880, 884)...
14     {'Date': {'count': 1, 'span_info': [(880, 884)...
15                                                    {}
16                                                    {}
17                             

In [9]:
df["text_clean"]

0      Contract notice  Afc 2019 and cross cutting sk...
1      Contract notice  Innovation partnership for th...
2      Contract notice  Long term rental of coaches w...
3      Contract notice  Hosting of the is of the anac...
4      Contract notice  Supply of signaling devices a...
5      Contract notice  Project management for rehabi...
6      Tenders are invited for Tenants ordering manag...
7      Tenders are invited for Creation of a graphic ...
8      L essentiel du marché \t\t\t \tNom et adresse ...
9                          Détail de la consultation ...
10                         Détail de la consultation ...
11                         Détail de la consultation ...
12                         Détail de la consultation ...
13                         Détail de la consultation ...
14                         Détail de la consultation ...
15     Électroménager et de matériel audiovisuel.Tele...
16     Fourniture et livraison de produits d'entretie...
17     Fourniture de dispositif

In [10]:
## détecter la langue du description
from langdetect import detect
df["lang_des"] = df["text_clean"].apply(detect)

In [11]:
offres_fr = df[df.lang_des == "fr"]
offres_en = df[df.lang_des == "en"]

In [12]:
#### Problème : certaines descriptions contiennent à la fois du texte en anglais et en français 
offres_fr["text_clean"][2]

"Contract notice  Long term rental of coaches without driver  The purpose of this consultation is the long term rental of driverless coaches for the needs of municipal services.  This contract is divided into lots  no  Deadline for receipt of tenders or requests to participate  September 25 2020   4 00 pm  Language  s  official  s  of the EU  French  Avis No 20 63451   publié le 08 08 2020   Dépt.   92 Mairie de nanterre   Location longue durée d'autocars sans chauffeur   Avis de marché Procédure Ouverte        Références complètes&nbsp     Avis  20 63451        Référence de TED     non disponible         Annonce            Département s  de publication   92Annonce No 20 63451I.II.III.IV.VI.AVIS DE MARCHÉDirective 2014 24 UELe présent avis constitue un appel à la concurrence Section I   Pouvoir adjudicateur                   I.1       NOM ET ADRESSES                Mairie de Nanterre 88 110 rue du 8 mai 1945 92000 Nanterre F Courriel   service.commandepublique mairie nanterre.fr  Code 

In [13]:
### texte en anglais
vec=offres_en["text_clean"]
vec2=vec.to_list()
vec2

test=list()
for doc in nlpsm.pipe(vec2, disable=["tagger", "parser"]):
    test.append([(ent.text, ent.label_) for ent in doc.ents])

In [15]:
## texte en français
text = offres_fr["text_clean"][4]
doc = nlpFR(text)
for entity in doc.ents:
    print(entity.text, entity.label_)

Contract MISC
Supply of signaling devices and urban comfort and safety equipment   MISC
The purpose of the contract that is the subject of this consultation is the supply of signaling devices and urban comfort and safety equipment necessary for the services of the city of nanterre MISC
They are MISC
It includes in particular the supply of vertical and permanent signage MISC
This contract is divided into lots MISC
Deadline for receipt of tenders or requests to participate MISC
September ORG
Language  s  official  s  of the EU MISC
French  Avis MISC
Dépt MISC
Avis de marché ORG
Avis ORG
Référence de TED MISC
MARCHÉDirective MISC
UELe MISC
Section I    PER
ADRESSES                Mairie LOC
Nanterre LOC
Nanterre F LOC
Adresse principale   http MISC
AccueilEntreprise MISC
I.2 MISC
AccueilEntreprise MISC
AccueilEntreprise ORG
ADJUDICATEUR ORG
Autorité régionale ORG
I.5 MISC
Services généraux ORG
Section II   Objet                   II.1       ÉTENDUE ORG
II.1.1 MISC
Fourniture LOC


In [16]:
displacy.serve(doc, style="ent")

/Users/marwa/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
##


In [29]:
text = offres_fr["text_clean"][4]
doc = nlpMulti("Rasmus Luckow-Nielsen is a good man")

Rasmus Rasmus
Luckow Luckow
- -
Nielsen Nielsen
is is
a a
good good
man man


In [ ]:

)